# Test server

In [3]:
from function.TickDataServerTest import *
import zmq

During the while loop, a new instrument price gets simulated, and a message is created, printed, and sent via the socket.

# Price Sending

In [5]:
ip = InstrumentPrice()

while True:
    msg = '{} {:.2f}'.format(ip.symbol, ip.simulate_value())
    print(msg)
    socket.send_string(msg)
    time.sleep(random.random() * 2)

SYMBOL 100.00
SYMBOL 99.97
SYMBOL 100.20
SYMBOL 99.99
SYMBOL 99.99
SYMBOL 100.33
SYMBOL 100.81
SYMBOL 100.32
SYMBOL 100.52
SYMBOL 100.77
SYMBOL 100.31
SYMBOL 100.41
SYMBOL 100.48
SYMBOL 100.18
SYMBOL 100.14
SYMBOL 100.30
SYMBOL 100.33
SYMBOL 100.36
SYMBOL 99.89
SYMBOL 99.72
SYMBOL 99.34
SYMBOL 99.29
SYMBOL 98.79
SYMBOL 98.86
SYMBOL 98.63
SYMBOL 98.90
SYMBOL 98.91
SYMBOL 98.63
SYMBOL 99.09
SYMBOL 99.15
SYMBOL 98.94
SYMBOL 97.86
SYMBOL 98.11
SYMBOL 97.57
SYMBOL 97.30
SYMBOL 97.41
SYMBOL 97.17
SYMBOL 96.63
SYMBOL 96.53
SYMBOL 96.48
SYMBOL 96.76
SYMBOL 96.35
SYMBOL 96.52
SYMBOL 96.41
SYMBOL 97.19
SYMBOL 97.10
SYMBOL 96.99
SYMBOL 97.54
SYMBOL 97.50
SYMBOL 97.36
SYMBOL 96.99
SYMBOL 97.36
SYMBOL 97.22
SYMBOL 97.17
SYMBOL 96.55
SYMBOL 96.81
SYMBOL 97.04
SYMBOL 97.00
SYMBOL 97.02
SYMBOL 97.08
SYMBOL 96.30
SYMBOL 96.00
SYMBOL 96.33
SYMBOL 96.22
SYMBOL 96.47
SYMBOL 96.30
SYMBOL 96.21
SYMBOL 95.55
SYMBOL 95.85
SYMBOL 96.01
SYMBOL 96.38
SYMBOL 96.56
SYMBOL 96.47
SYMBOL 96.61
SYMBOL 96.38
SYMBOL 95.

KeyboardInterrupt: 

# Sample Bar Data

In [ ]:
import zmq
import math
import time
import random

context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind('tcp://0.0.0.0:5556')

for _ in range(10):
# while True:
    bars = [random.random() * 100 for _ in range(8)]
    msg = ' '.join([f'{bar:.3f}' for bar in bars])
    print(msg)
    socket.send_string(msg)
    time.sleep(random.random() * 2)

In [ ]:
import zmq

# This line of code defines the so-called channel to which the socket subscribes. 
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

while True:
    data = socket.recv_string()
    print(data)

In [ ]:
import zmq
import datetime
import numpy as np
import pandas as pd

context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

df = pd.DataFrame()
mom = 3
min_length = mom + 1

while True:
    data = socket.recv_string()
    t = datetime.datetime.now()
    sym, value = data.split()
    df = df.append(pd.DataFrame({sym: float(value)}, index=[t]))
    dr = df.resample('5s', label='right').last()
    dr['returns'] = np.log(dr / dr.shift(1))
    if len(dr) > min_length:
        min_length += 1
        dr['momentum'] = np.sign(dr['returns'].rolling(mom).mean())
        print('\n' + '=' * 51)
        print('NEW SIGNAL | {}'.format(datetime.datetime.now()))
        print('=' * 51)
        print(dr.iloc[:-1].tail())
        if dr['momentum'].iloc[-2] == 1.0:
            print('\nLong market position.')
            # take some action (e.g., place buy order)
        elif dr['momentum'].iloc[-2] == -1.0:
            print('\nShort market position.')
            # take some action (e.g., place sell order)

In [ ]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

symbol = 'SYMBOL'

fig = go.FigureWidget()  
fig.add_scatter()  
fig

context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

times = list()
prices = list()

for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()  
    times.append(t)  
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times  
    fig.data[0].y = prices  

In [ ]:
while True:
    bars = [random.random() * 100 for _ in range(8)]
    msg = ' '.join([f'{bar:.3f}' for bar in bars])
    print(msg)

# Vietnam Market Pull Data

In [ ]:
import asyncio
import websockets
import json
import zmq

# Setup ZMQ Publisher
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind('tcp://0.0.0.0:5555')  # Hoặc IPC tùy hệ thống

async def stream_tvsi():
    uri = "wss://tvsi-iboard.tvsi.com.vn/ws"
    async with websockets.connect(uri) as ws:
        await ws.send(json.dumps({"sub": ["FPT", "VCB"]}))
        for _ in range(50):
        # while True:
            msg = await ws.recv()
            socket.send_string(msg)  # Gửi dữ liệu tới các SUB client

await stream_tvsi()

In [ ]:
import requests

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://iboard.tvsi.com.vn/",
}

res = requests.get(
    "https://price-api.tvsi.com.vn/history/intraday?symbol=FPT&date=2024-04-08",
    headers=headers
)
data = res.json()

In [ ]:
print(data.head())

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_fpts_price_history(symbol):
    url = f"https://www.fpts.com.vn/Portal/Stock/StocksDetail.aspx?code={symbol}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    table = soup.find("table", {"id": "StockDetail1_rptData"})
    if table is None:
        print(f"Không tìm thấy bảng cho {symbol}")
        return pd.DataFrame()

    df = pd.read_html(str(table))[0]
    return df

df_fpt = get_fpts_price_history("FPT")
df_vcb = get_fpts_price_history("VCB")

print("FPT:")
print(df_fpt.head())

print("VCB:")
print(df_vcb.head())


Không tìm thấy bảng cho FPT
Không tìm thấy bảng cho VCB
FPT:
Empty DataFrame
Columns: []
Index: []
VCB:
Empty DataFrame
Columns: []
Index: []


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import tempfile
import time

def get_ssi_price_history(symbol):
    url = f"https://iboard.ssi.com.vn/bang-gia/co-phieu/{symbol}"

    # Khởi tạo Chrome profile độc lập tạm thời
    temp_profile = tempfile.mkdtemp()

    options = Options()
    options.add_argument(f'--user-data-dir={temp_profile}')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')  # Có thể bỏ nếu muốn xem giao diện
    service = Service()

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(10)

    try:
        driver.find_element(By.XPATH, '//div[contains(text(), "Lịch sử giá")]').click()
        time.sleep(3)
    except:
        pass

    table = driver.find_element(By.CLASS_NAME, 'ant-table-content')
    html = table.get_attribute("innerHTML")
    df = pd.read_html(f"<table>{html}</table>")[0]

    driver.quit()
    return df

df_fpt = get_ssi_price_history("FPT")
print(df_fpt.head())


In [9]:
# pip install investpy-lite
import investpy

df = investpy.get_stock_historical_data(stock='FPT', country='vietnam', from_date='01/03/2024', to_date='08/04/2024')
print(df.head())


ConnectionError: ERR#0015: error 403, try again later.